In [11]:
# Preliminaries: Teach Julia that functions form a vector space
import Base.+,Base.*,Base.zero
+(f::Function,g::Function) = x -> f(x)+g(x)
*(c::Number,f::Function) = x -> c*f(x)
*(f::Function,c::Number) = x -> c*f(x)
zero(Function) = x -> 0

zero (generic function with 18 methods)

## Examples of  Linear Transformations 

Some operations are fairly obviously linear.  No basis is needed to see this.  It is
efficient theoretically to treat these operations in one fell swoop in a unified way.

For example the derivative of functions $f(x)$ is obviously linear. Derivatives of sums are sums of derivatives: (f+g)'=f'+g'. Derivatives of constant multiples are constant multiples of derivatives (cf)'=cf'.
Another function transformation example that is obviously linear is the shift by a constant a: $f(x) \rightarrow f(x+a):$



In [21]:
function T(f::Function)
    return x->f(x+1)
end
    

T (generic function with 1 method)

In [22]:
[T(sin)(5) sin(6)]

1×2 Array{Float64,2}:
 -0.279415  -0.279415

In [25]:
# An example check that shifting is linear
# we check at x=5 that 2*T(sin)+3*T(cos) = T(2*sin+3*cos), where T denotes shift by one
[( 2*T(sin) + 3*T(cos) )(5) T( 2*sin + 3*cos )(5)]

1×2 Array{Float64,2}:
 2.32168  2.32168

Another example considers the vector space of $m_1 \times n_1$ matrices $X$.  If we take a constant
$m_2 \times m_1$ matrix $B$ and a constant $n_2 \times n_1$ matrix $A$ then the map $X \rightarrow BXA^T$ is
obviously a linear map from a vector space of dimension $m_1n_1$ to a vector space of dimension $m_2n_2$.
(Check: $ B(c_1 X+c_2 Y)A^T=   c_1 BXA^T + c_2 BYA^T$.)

## Example 1: Derivatives of  (a cos x + b sin x) 
Consider the 2 dimensional vector space consisting of linear combinations of "sine" and "cosine". How can we take the derivative of a function in this vector space?

### Derivatives: Method 1, symbolically.  Matches the paper and pencil method closely.

In [28]:
using SymPy

In [29]:
x,a,b = Sym.(["x","a","b"]);

In [42]:
diff( a*cos(x) + b*sin(x) ,x)

-a*sin(x) + b*cos(x)

### Method 2, matrix-vector.  Emphasizes the linear nature of derivatives. (Easy to imagine a numerical implementation.)

$\begin{pmatrix} a' \\b' \end{pmatrix} = 
\begin{pmatrix} 0 & 1 \\-1 & 0 \end{pmatrix}
\begin{pmatrix} a \\b \end{pmatrix}$

Understanding: $\begin{pmatrix} a' \\b' \end{pmatrix}$ is shorthand for
$a\cos x + b\sin x$.

Example: Differentiate $f(x)=5\cos x + 2 \sin x$:
1. Encode  f(x) for computation as the vector  $\begin{pmatrix} 5 \\ 2 \end{pmatrix}.$
2. Apply $\begin{pmatrix} 0 & 1 \\-1 & 0 \end{pmatrix}$
to $\begin{pmatrix} 5 \\ 2 \end{pmatrix}$
yielding $\begin{pmatrix} 2 \\ -5 \end{pmatrix}.$
3. Decode $\begin{pmatrix} 2 \\ -5 \end{pmatrix}$ as
$2 \cos x -5 \sin x.$

###  Method 3: no shorthand.  Combines method 1 and method 2.

$\frac{d}{dx} \begin{pmatrix} \sin x& \cos x \end{pmatrix}
\begin{pmatrix} a \\b \end{pmatrix}
=
\begin{pmatrix} \sin x& \cos x \end{pmatrix}
\begin{pmatrix} 0 & -1 \\1 & 0 \end{pmatrix}
\begin{pmatrix} a \\b \end{pmatrix}
$

Method 2 is purely numerical.  The interpretation is imposed by
the human.  Method 3 can be interpreted as method 2 (matrix times vector) with the labels. 

If one associates differently, Method 3 can be interpeted as knowing
the derivative on the basis functions is sufficient for knowing
this linear transformation everywhere:

$\frac{d}{dx} \begin{pmatrix} \sin x& \cos x \end{pmatrix}
=
\begin{pmatrix} \sin x& \cos x \end{pmatrix}
\begin{pmatrix} 0 & -1 \\1 & 0 \end{pmatrix}
$

### Observation:  
Method 1 is straightforward but in the end
bulky.  Method 2 shows that the linear transformation defined
by differentiating can be encoded as a simple matrix times vector,
which is very efficient on a computer, and also gets to the algebraic heart of the operation.  Method 3 organizes the symbolic with the matrices in a way that points to the generalization.
<br>
Most students of calculus learn that differentiation is linear.
Derivatives of sums are sums of derivatives ,(f+g)'=f'+g'.  Derivatives of
constant multiples are constant multiples of derivatives (cf)'=cf'.

### With code:

In [76]:
f=([sin cos]*[0 1;-1 0]*[5,2])[1]
x=rand()
[f(x) 2sin(x)-5cos(x)] ## Check that it gives the right function

1×2 Array{Float64,2}:
 -1.29521  -1.29521

## In general

If $v_1,\ldots,v_n$ is a basis for a vector space $V$, 
and <br> $w_1,\ldots,w_m$ is a basis for a vector space $W$,
and $T$ is some linear transformation,
we can write

$$ T[v_1,\ldots,v_n]
\begin{pmatrix} c_1 \\ \vdots \\ c_n \end{pmatrix}
=
[w_1,\ldots,w_m] * A* \begin{pmatrix} c_1 \\ \vdots  \\ c_n \end{pmatrix}$$
for some $m \times n$ matrix $A$.

One can associate the equation above concentrating
on
$$ T[v_1,\ldots,v_n]
=
[w_1,\ldots,w_m] * A$$


to think of
$T$ as applied to every basis vector of $V$ to get
some linear combination of the basis vectors of $W$,
or one can do "Method 2" and think of
$\begin{pmatrix} c_1 \\ \vdots \\ c_n \end{pmatrix}$
as the coefficients in the basis for $V$, and 
$A\begin{pmatrix} c_1 \\ \vdots \\ c_n \end{pmatrix}$
as the cooeficients of $Tv$ in the basis for $W$.

## Example 2:  Shifting (a cos x + b sin x)

Convince yourself without matrices that
$Tf$ defined by $ (Tf)(x)=f(x+\theta)$ is linear for
any constant $\theta$.

With matrices we have

$T  \begin{pmatrix} \sin x& \cos x \end{pmatrix}
=
\begin{pmatrix} \sin x& \cos x \end{pmatrix}
\begin{pmatrix} \cos \theta & -\sin \theta  \\ \sin \theta & \cos \theta \end{pmatrix}
$
or
$T  \begin{pmatrix} \sin x& \cos x \end{pmatrix}
\begin{pmatrix} a \\b \end{pmatrix}
=
\begin{pmatrix} \sin x& \cos x \end{pmatrix}
\begin{pmatrix} \cos \theta & -\sin \theta  \\ \sin \theta & \cos \theta \end{pmatrix}
\begin{pmatrix} a \\b \end{pmatrix}
$


which can be done symbolically but gets a little messier looking.  The linear algebra is just tidier.

In [87]:
x = Sym("x")
f = a*sin(x) + b*cos(x)
Tf = subs(f,x,x+Sym("theta"))

a*sin(theta + x) + b*cos(theta + x)

In [90]:
expand_trig(Tf)

a*(sin(theta)*cos(x) + sin(x)*cos(theta)) + b*(-sin(theta)*sin(x) + cos(theta)
*cos(x))

Of course Example 1 is a special case of Example 2 because
the derivative is the same as shifting by $\pi/2$ on this very special  vector space.

## Example 3. Change of basis for polynomials

Suppose one wants to work with Laguerre polynomials.
Wikipidia can supply the first few of these for us:
 [Laguerre up to degree 6](https://en.wikipedia.org/wiki/Laguerre_polynomials#The_first_few_polynomials).

In [85]:
A = Rational.([
 1   1   2    6  24
 0  -1  -4  -18 -96
 0   0   1    9  72
 0   0   0   -1 -16
 0   0   0    0   1])./[1 1 2 6 24]

5×5 Array{Rational{Int64},2}:
 1//1   1//1   1//1   1//1   1//1 
 0//1  -1//1  -2//1  -3//1  -4//1 
 0//1   0//1   1//2   3//2   3//1 
 0//1   0//1   0//1  -1//6  -2//3 
 0//1   0//1   0//1   0//1   1//24

Check from the Wikipedia article that
$[L_0 \ L_1 \ L_2 \ L_3 \ L_4]=[1 \ x \ x^2 \ x^3 \ x^4] * A$

In [86]:
[1 x x^2 x^3 x^4]*A

1×5 Array{SymPy.Sym,2}:
 1  -x + 1  x^2/2 - 2*x + 1  …  x^4/24 - 2*x^3/3 + 3*x^2 - 4*x + 1

Convince yourself that to obtain
the coefficients of 
$c_0 L_0 + c_1 L_1 + c_2 L_2 + c_3 L_3$
in the standard basis $1,x,x^2,x^3$
one must simply compute
$A * \begin{pmatrix} c_0 \\ c_1 \\ c_2 \\ c_3 \end{pmatrix}.$

<br>

Notationally, we are saying that <br>
$[L_0 \ L_1 \ L_2 \ L_3]*\begin{pmatrix} c_0 \\ c_1 \\ c_2 \\ c_3 \end{pmatrix}=[1 \ x \ x^2 \ x^3] * A*\begin{pmatrix} c_0 \\ c_1 \\ c_2 \\ c_3 \end{pmatrix}$

Of course inv(A) let's us go the other way
(from monomial coefficients to Laguerre coefficients)

In [103]:
Int.(inv(A))

5×5 Array{Int64,2}:
 1   1   2    6   24
 0  -1  -4  -18  -96
 0   0   2   18  144
 0   0   0   -6  -96
 0   0   0    0   24

Thus for example <br>
$x^3 = 6(L_0 - 3 L_1 + 3 L_2 - 1 L_3).$

Note: the numbers are pascal's triangle times factorials

What if we want to differentiate quartics written in a Laguerre polynomial basis but we only know how to differentiate in a monomial basis?
<br>
In the standard basis $1,x,x^2,x^3,x^4$ the derivative is this matrix:

In [89]:
D=[0 1 0 0 0
   0 0 2 0 0
   0 0 0 3 0
   0 0 0 0 4
   0 0 0 0 0]

5×5 Array{Int64,2}:
 0  1  0  0  0
 0  0  2  0  0
 0  0  0  3  0
 0  0  0  0  4
 0  0  0  0  0

 We claim
$\frac{d}{dx} [1 \ x \ x^2 \ x^3 \ x^4] = [1 \  x \  x^2 \ x^3 \ x^4]*D$

In [90]:
[1 x x^2 x^3 x^4]

1×5 Array{SymPy.Sym,2}:
 1  x  x^2  x^3  x^4

In [91]:
[1 x x^2 x^3 x^4]*D

1×5 Array{SymPy.Sym,2}:
 0  1  2*x  3*x^2  4*x^3

In [94]:
## Now the derivative in a Laguerre Basis
Int.(A\D*A)

5×5 Array{Int64,2}:
 0  -1  -1  -1  -1
 0   0  -1  -1  -1
 0   0   0  -1  -1
 0   0   0   0  -1
 0   0   0   0   0

That's interesting.  The pattern seems to suggest that
<br>
$\frac{d}{dx}L_k(x) = -\sum_{j=0}^{k-1}L_j(x)$
which is a true identity.

[The wikipedia article](https://en.wikipedia.org/wiki/Laguerre_polynomials) states
right after the words Sheffer sequence that
$$\frac{d}{dx} L_n = -L_{n-1} + \frac{d}{dx} L_{n-1}$$
which you should recognize states that the $n$th column
of the matrix $A^{-1}DA$ is the same as the $n-1$st column,
with an extra $-1$ in the $(n-1)$st entry.

In [120]:
# Not working yet
# SymPy.mpmath[:laguerre](4,0,Sym("x")) 

# Change of Basis
The above example shows how the derivative matrix can look when changing basis.

In [106]:
# Derivative in standard basis
D

5×5 Array{Int64,2}:
 0  1  0  0  0
 0  0  2  0  0
 0  0  0  3  0
 0  0  0  0  4
 0  0  0  0  0

In [107]:
# Derivative in Laguerre basis
Int.(A\D*A)

5×5 Array{Int64,2}:
 0  -1  -1  -1  -1
 0   0  -1  -1  -1
 0   0   0  -1  -1
 0   0   0   0  -1
 0   0   0   0   0

Conclusion: Similar matrices represent the same linear transformation in a different basis

## Example 3: Kronecker Products

Let's check that the [Kronecker Product](https://en.wikipedia.org/wiki/Kronecker_product) is the matrix
for the transformation $X\rightarrow BXA^T$

In [108]:
A = rand(1:9,4,4)

4×4 Array{Int64,2}:
 2  3  2  4
 8  7  1  4
 5  9  7  3
 1  1  2  6

In [109]:
B = rand(1:9,4,4)

4×4 Array{Int64,2}:
 2  9  8  6
 8  2  2  9
 2  8  6  6
 4  1  4  9

In [110]:
kron(A,B)

16×16 Array{Int64,2}:
  4  18  16  12   6  27  24  18   4  18  16  12   8  36  32  24
 16   4   4  18  24   6   6  27  16   4   4  18  32   8   8  36
  4  16  12  12   6  24  18  18   4  16  12  12   8  32  24  24
  8   2   8  18  12   3  12  27   8   2   8  18  16   4  16  36
 16  72  64  48  14  63  56  42   2   9   8   6   8  36  32  24
 64  16  16  72  56  14  14  63   8   2   2   9  32   8   8  36
 16  64  48  48  14  56  42  42   2   8   6   6   8  32  24  24
 32   8  32  72  28   7  28  63   4   1   4   9  16   4  16  36
 10  45  40  30  18  81  72  54  14  63  56  42   6  27  24  18
 40  10  10  45  72  18  18  81  56  14  14  63  24   6   6  27
 10  40  30  30  18  72  54  54  14  56  42  42   6  24  18  18
 20   5  20  45  36   9  36  81  28   7  28  63  12   3  12  27
  2   9   8   6   2   9   8   6   4  18  16  12  12  54  48  36
  8   2   2   9   8   2   2   9  16   4   4  18  48  12  12  54
  2   8   6   6   2   8   6   6   4  16  12  12  12  48  36  36
  4   1   4   9   

In [112]:
X = rand(1:9,4,4)

4×4 Array{Int64,2}:
 6  2  7  2
 5  2  8  4
 6  1  4  5
 4  4  5  3

In [115]:
# The vec operator strings a matrix column wise into one long column
[ kron(A,B)*vec(X)  vec(B*X*A')]

16×2 Array{Int64,2}:
 1108  1108
  880   880
  974   974
  758   758
 1950  1950
 1623  1623
 1714  1714
 1395  1395
 2461  2461
 2110  2110
 2186  2186
 1751  1751
 1067  1067
  780   780
  930   930
  687   687

You might check that kron(A,B) is the matrix of the linear transformation $X \rightarrow BXA^T$
in the following basis:

In [119]:
for j=1:4, i=1:4
    V=zeros(Int,4,4)
    V[i,j]=1
    display(V)
end

4×4 Array{Int64,2}:
 1  0  0  0
 0  0  0  0
 0  0  0  0
 0  0  0  0

4×4 Array{Int64,2}:
 0  0  0  0
 1  0  0  0
 0  0  0  0
 0  0  0  0

4×4 Array{Int64,2}:
 0  0  0  0
 0  0  0  0
 1  0  0  0
 0  0  0  0

4×4 Array{Int64,2}:
 0  0  0  0
 0  0  0  0
 0  0  0  0
 1  0  0  0

4×4 Array{Int64,2}:
 0  1  0  0
 0  0  0  0
 0  0  0  0
 0  0  0  0

...

4×4 Array{Int64,2}:
 0  0  0  0
 0  0  0  0
 0  0  0  0
 0  0  1  0

4×4 Array{Int64,2}:
 0  0  0  1
 0  0  0  0
 0  0  0  0
 0  0  0  0

4×4 Array{Int64,2}:
 0  0  0  0
 0  0  0  1
 0  0  0  0
 0  0  0  0

4×4 Array{Int64,2}:
 0  0  0  0
 0  0  0  0
 0  0  0  1
 0  0  0  0

4×4 Array{Int64,2}:
 0  0  0  0
 0  0  0  0
 0  0  0  0
 0  0  0  1